## Choosing the best locations for pickup lockers

In [1]:
import neo4j
import math
import numpy as np
import pandas as pd
import re
from sklearn.neighbors import BallTree


import psycopg2
from scipy import spatial
from geographiclib.geodesic import Geodesic

### Necessary helper functions for the coding. This is from the labs, exercises, with our modifications

In [2]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)

def my_neo4j_run_query_pandas(query, **kwargs):
    "run a query and return the results in a pandas dataframe"
    
    result = session.run(query, **kwargs)
    
    df = pd.DataFrame([r.values() for r in result], columns=result.keys())
    
    return df

def my_neo4j_create_customer_node(customer_id, zip_code):
    "create a node with label Station"
    
    query = """
    
    CREATE (:Customer {
                        customer_id: $customer_id,
                        zip_code: $zip_code
                    })
    
    """
    
    session.run(query, customer_id=customer_id, zip_code = zip_code)

def create_relationship_one_way_customer_station(from_customer, to_station, weight):
    "create a relationship one way between two stations with a weight"
    
    query = """
    
    MATCH (from:Customer), 
          (to:Station)
    WHERE from.customer_id = $from_customer and to.name = $to_station
    CREATE (from)-[:DIST {weight: $weight}]->(to)
    
    """
    
    session.run(query, from_customer=from_customer, to_station=to_station, weight=weight)
    
def my_neo4j_number_nodes_relationships():
    "print the number of nodes and relationships"
   
    
    query = """
        match (n) 
        return n.name as node_name, labels(n) as labels
        order by n.name
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_nodes = df.shape[0]
    
    
    query = """
        match (n1)-[r]->(n2) 
        return n1.name as node_name_1, labels(n1) as node_1_labels, 
            type(r) as relationship_type, n2.name as node_name_2, labels(n2) as node_2_labels
        order by node_name_1, node_name_2
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    number_relationships = df.shape[0]
    
    print("-------------------------")
    print("  Nodes:", number_nodes)
    print("  Relationships:", number_relationships)
    print("-------------------------")

def wipeout_customers():
    query = """
        Match (n:Customer)
        DETACH Delete n
    """
    session.run(query)

### Connections to postgres and neo4j

In [3]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

driver = neo4j.GraphDatabase.driver(uri="neo4j://neo4j:7687", auth=("neo4j","ucb_mids_w205"))
session = driver.session(database="neo4j")

wipeout_customers()

### Get the stations list along with their coordinates

In [4]:
rollback_before_flag = True
rollback_after_flag = True

query = """

SELECT *
FROM stations

"""

stations = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
stations.head()

,station,latitude,longitude,transfer_time
0,12th Street,37.803608,-122.272006,282
1,16th Street Mission,37.764847,-122.420042,287
2,19th Street,37.807869,-122.268980,67
3,24th Street Mission,37.752000,-122.418700,277
4,Antioch,37.996281,-121.783404,0


### Getting zip codes with their coordinates

In [5]:
rollback_before_flag = True
rollback_after_flag = True

query = """

SELECT *
FROM customers

"""

zip_codes = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
zip_codes

,customer_id,first_name,last_name,street,city,state,zip,closest_store_id,distance
0,1,Robb,Weaving,5 Ramsey Place,Oakland,CA,94609,1,1
1,2,Robby,Belliard,6 Londonderry Plaza,Oakland,CA,94609,1,1
2,3,Sadella,Caudrelier,548 Mcguire Parkway,Oakland,CA,94609,1,1
3,4,Holmes,Shimmings,99 Kennedy Court,Oakland,CA,94609,1,1
4,5,Beverley,Gubbin,51 Mcbride Drive,Oakland,CA,94609,1,1
...,...,...,...,...,...,...,...,...,...
31077,31078,Hugo,Domeney,529 5th Plaza,Thompsons Station,TN,37179,5,25
31078,31079,Glenn,Putson,1347 Westend Crossing,Thompsons Station,TN,37179,5,25
31079,31080,Minnie,Antham,9 Judy Place,Thompsons Station,TN,37179,5,25
31080,31081,Linet,Djorvic,29 Trailsway Drive,Thompsons Station,TN,37179,5,25


### Getting customers' coordinates

In [6]:
query = """

SELECT customer_id, customers.zip, latitude, longitude
FROM customers, zip_codes 
WHERE customers.zip = zip_codes.zip

"""

customer_loc = my_select_query_pandas(query, rollback_before_flag, rollback_after_flag)
customer_loc

,customer_id,zip,latitude,longitude
0,1,94609,37.8343,-122.2643
1,2,94609,37.8343,-122.2643
2,3,94609,37.8343,-122.2643
3,4,94609,37.8343,-122.2643
4,5,94609,37.8343,-122.2643
...,...,...,...,...
31077,31078,37179,35.8116,-86.9277
31078,31079,37179,35.8116,-86.9277
31079,31080,37179,35.8116,-86.9277
31080,31081,37179,35.8116,-86.9277


## Building customers nodes, and their relations to their closest stations

We use KDTree to quickly identify the closest stations to our customers

In [7]:

bt = BallTree(np.deg2rad(stations[['latitude', 'longitude']].values), metric='haversine')
dists, inds = bt.query(customer_loc[['latitude', 'longitude']], k = 1)

#convert distances to miles and flatten multi-level indices 
dists = dists.flatten()
inds = inds.flatten()

dists = dists * 3958.8 

customer_loc['distance'] = dists * 3958.8 
customer_loc['station'] = stations.iloc[inds]['station'].values

### Create **Customer** nodes

In [8]:
connection.rollback()
cust_ids, zip_codes = customer_loc['customer_id'].values, customer_loc['zip'].values
for i in range(len(cust_ids)):
    my_neo4j_create_customer_node(cust_ids[i], zip_codes[i])

In [9]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 31296
  Relationships: 652
-------------------------


Create **relationship**

In [10]:
connection.rollback()
for i in range(len(dists)):
    create_relationship_one_way_customer_station(cust_ids[i], 'arrive ' + customer_loc['station'].values[i], dists[i])

In [11]:
my_neo4j_number_nodes_relationships()

-------------------------
  Nodes: 31296
  Relationships: 31734
-------------------------


### Run centrality function to identify the optimal stations to setup lockers

Here we choose to run betweenness and degree centrality to identify the optimal locations for the pickup lockers.
Our goal is to pick the locations that have the balance of close to many customers, and also have many traffic from stations to stations to them.
Our methodology is choosing the top 20 of each, then take the join of them.

In [12]:
def neo4j_betweeness(top):
    query = "CALL gds.graph.drop('ds_graph', false)"
    session.run(query)

    query = "CALL gds.graph.project('ds_graph', ['Station', 'Customer'], ['LINK', 'DIST'], {relationshipProperties: 'weight'})"
    session.run(query)
    
    query = """
        CALL gds.betweenness.stream('ds_graph')
        YIELD nodeId, score
        WITH gds.util.asNode(nodeId).name as originalName, score
        WITH trim(reduce(s = '', part IN tail(split(originalName, ' ')) | s + part + ' ')) AS name, score
        RETURN name, avg(score) AS avgScore
        ORDER BY avgScore DESC
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    return df.head(top)

def neo4j_degree(top):
    query = "CALL gds.graph.drop('ds_graph', false)"
    session.run(query)

    query = "CALL gds.graph.project('ds_graph', ['Station', 'Customer'], ['LINK', 'DIST'])"
    session.run(query)
    
    query = """
        CALL gds.degree.stream('ds_graph')
        YIELD nodeId, score
        WITH gds.util.asNode(nodeId).name as originalName, score
        WITH trim(reduce(s = '', part IN tail(split(originalName, ' ')) | s + part + ' ')) AS name, score
        RETURN name, avg(score) AS avgScore
        ORDER BY avgScore DESC
    """
    
    df = my_neo4j_run_query_pandas(query)
    
    return df.head(top)

In [13]:
connection.rollback()
betweeness_stations = neo4j_betweeness(20)

In [14]:
connection.rollback()
degree_stations = neo4j_degree(20)

In [15]:
bs = set(betweeness_stations['name'])
ds = set(degree_stations['name'])

In [16]:
bs

{'12th Street',
 '16th Street Mission',
 '19th Street',
 '24th Street Mission',
 'Bay Fair',
 'Civic Center',
 'Coliseum',
 'Embarcadero',
 'Fruitvale',
 'Hayward',
 'Lafayette',
 'Lake Merritt',
 'MacArthur',
 'Montgomery Street',
 'Orinda',
 'Powell Street',
 'Rockridge',
 'San Leandro',
 'Walnut Creek',
 'West Oakland'}

In [17]:
ds

{'12th Street',
 '16th Street Mission',
 '19th Street',
 '24th Street Mission',
 'Ashby',
 'Balboa Park',
 'Bay Fair',
 'Civic Center',
 'Coliseum',
 'Colma',
 'Daly City',
 'Embarcadero',
 'Fruitvale',
 'Glen Park',
 'Lake Merritt',
 'MacArthur',
 'Montgomery Street',
 'Powell Street',
 'San Leandro',
 'West Oakland'}

### Finding the join between two sets

In [18]:
optimal_stations = bs.intersection(ds)
optimal_stations

{'12th Street',
 '16th Street Mission',
 '19th Street',
 '24th Street Mission',
 'Bay Fair',
 'Civic Center',
 'Coliseum',
 'Embarcadero',
 'Fruitvale',
 'Lake Merritt',
 'MacArthur',
 'Montgomery Street',
 'Powell Street',
 'San Leandro',
 'West Oakland'}